# Exploring Effects of Order Dependency in LLMs

While LLMs have demonstrated remarkable capabilities in understanding and generating text, researchers have found they also exhibit a number of unexpected behavioral patterns. The *order dependency problem* refers to LLMs' well-documented sensitivity to the order information is presented in a prompt. Making small changes to the order of tokens in an LLM's input sequence has been shown to generate large changes in the model's predicted outputs.

One specific area where this has been thoroughly studied is in answering multiple choice questions (MCQs). Pezeshkpour and Hruschka (2023) and Zheng et al. (2024) showed that an LLMs' performance on MCQ tasks can be dramatically altered simply by changing the order of the choices for each question. This is particularly concerning given that MCQ tasks are widely used in benchmarks such as MMLU (Hendrycks et al. 2021).

**TODO:** discuss implications and risks of order dependency problem

**TODO:** highlight a sample of solutions

**TODO:** roadmap the rest of the post

# Roadmap

* Dataset
* Demonstrate positional bias
* Improve positional bias
* Discussion